In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

sind_strs = [ 'S01', 'S02', 'S03', 'S04', 'S05', 'S07' ]

In [ ]:
#sind_strs = [ 'S01', 'S02', 'S03', 'S04', 'S05', 'S07' ]

import globvars as gv
from os.path import join as pjoin
from utils import loadROILabels, loadSurfAndGrids
# it is very important to get the right order
roi_labels, srcgroups, source_coords = loadROILabels()
vis_info_per_subj = loadSurfAndGrids(sind_strs)
    
cgMNI = vis_info_per_subj['S_default']

from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj, RoiObj
from visbrain.gui import Brain

In [ ]:
cgMNI.keys()

##  plot S07 on Jan's brain surf

In [ ]:
import gc; gc.collect()
subj = 'S07'
#subj = 'S01'
vi = vis_info_per_subj[subj]

#xyz =  cgMNI['headsurfgrid_verts'].T * 100
xyz = vi['headsurfgrid_verts']
data = np.arange(xyz.shape[0])
radius_supp = 4
RADINFO = dict(radius_min=radius_supp, radius_max = radius_supp)

s_obj1  = SourceObj ('mysrc',xyz, data=data, cmap='inferno', **RADINFO)
s_obj2  = SourceObj ('mysrc',xyz, data=data, cmap='inferno', **RADINFO)

xyz2 = vi['headsurfgrid_mod_verts']
s_obj21  = SourceObj ('mysrc',xyz2, data=data, cmap='inferno', **RADINFO)
s_obj22  = SourceObj ('mysrc',xyz2, data=data, cmap='inferno', **RADINFO)


tc = True
verts = vi['headsurf_verts']
tris = vi['headsurf_tris']
b1 = BrainObj(subj, hemisphere='both' , translucent=tc, vertices=verts, faces=tris ) 
b2 = BrainObj(subj, hemisphere='both' , translucent=tc, vertices=verts, faces=tris ) 
b21 = BrainObj(subj, hemisphere='both' , translucent=tc, vertices=verts, faces=tris ) 
b22 = BrainObj(subj, hemisphere='both' , translucent=tc, vertices=verts, faces=tris ) 

verts = b1.vertices
#s_obj1.fit_to_vertices(verts) # result it shift

from visbrain.objects import BrainObj, SceneObj
bgcolor = 'black'
bgcolor = 'white'
sc = SceneObj(bgcolor=bgcolor, size=(400, 400))

#sc.add_to_subplot(b, row=0, col=0)
sc.add_to_subplot(b1, row=0, col=0, rotate='left', title=subj)
sc.add_to_subplot(b2, row=0, col=1, rotate='top', title=subj)
sc.add_to_subplot(s_obj1, row=0, col=0, rotate='left')
sc.add_to_subplot(s_obj2, row=0, col=1, rotate='top')

sc.add_to_subplot(b21,     row=1, col=0, rotate='left', title=subj)
sc.add_to_subplot(b22,     row=1, col=1, rotate='top', title=subj)
sc.add_to_subplot(s_obj21, row=1, col=0, rotate='left')
sc.add_to_subplot(s_obj22, row=1, col=1, rotate='top')

sc.preview()

## compare Jan's invidivdual grid with BEM

In [ ]:
import gc; gc.collect()
import mne
from plots import saveRenderVisBrainScene
subdir_fig = 'output/surf_consistency_check'

shift_xyz = 1
subj = 'S08'
#subj = 'S01'
vi = vis_info_per_subj[subj]
#%debug
# surf_type = 'pial' # 
# #surf_type = 'white'
# surf_type = 'inflated' # 
# surfnames = [f'lh.{surf_type}', f'rh.{surf_type}']
surfnames = ['brain.surf']
#%debug
print(surfnames)
bem_dir = pjoin('/home/demitau/data/subjects/',subj,'bem')

coords, faces = mne.read_surface(pjoin(bem_dir, 'inner_skull.surf'))
tc = False
b1 = BrainObj('', hemisphere='both' , translucent=tc, vertices=coords, faces=faces ) 
b2 = BrainObj('', hemisphere='both' , translucent=tc, vertices=coords, faces=faces ) 

verts = vi['headsurf_verts']
tris = vi['headsurf_tris']
b1_ = BrainObj(subj, hemisphere='both' , translucent=tc, vertices=verts, faces=tris ) 
b2_ = BrainObj(subj, hemisphere='both' , translucent=tc, vertices=verts, faces=tris ) 


verts = b1.vertices
maxs = np.max( verts, axis=0)
mins = np.min( verts, axis=0)
difs = maxs - mins
#s_obj1.fit_to_vertices(verts) # result it shift

xyz =  vi['headsurfgrid_verts'] * 10
maxs2 = np.max( xyz, axis=0)
mins2 = np.min( xyz, axis=0)
difs2 = maxs2 - mins2
if shift_xyz:
    xyz -= ( (maxs2+mins2)/2 )[None,:]
    xyz = xyz * ( (difs / difs2)[None,:] )
    xyz += ( (maxs+mins)/2 )[None,:]

data = np.arange(xyz.shape[0])
radius_supp = 4
RADINFO = dict(radius_min=radius_supp, radius_max = radius_supp)

s_obj1  = SourceObj ('mysrc',xyz, data=data, cmap='inferno', **RADINFO)
s_obj2  = SourceObj ('mysrc',xyz, data=data, cmap='inferno', **RADINFO)


from visbrain.objects import BrainObj, SceneObj
bgcolor = 'black'
#bgcolor = 'white'
sc = SceneObj(bgcolor=bgcolor, size=(400, 400))

#sc.add_to_subplot(b, row=0, col=0)
sc.add_to_subplot(b1,  row=0, col=0, zoom=2., rotate='left')
sc.add_to_subplot(b2,  row=0, col=1, zoom=2., rotate='top')
sc.add_to_subplot(b1_, row=1, col=0, zoom=2., rotate='left')
sc.add_to_subplot(b2_, row=1, col=1, zoom=2., rotate='top')

#sc.add_to_subplot(s_obj1, row=0, col=0, rotate='left')
#sc.add_to_subplot(s_obj2, row=0, col=1, rotate='top')

figfname_full= pjoin(gv.code_dir, subdir_fig, f'{subj}_surf_vs_bem.png')
render_result = saveRenderVisBrainScene(sc,figfname_full, render_only = 0)

#sc.preview()

In [ ]:
vb = Brain(brain_obj=b1, bgcolor='slateblue')
vb2 = Brain(brain_obj=b1_, bgcolor='slateblue')

In [ ]:
# if I do it after preview, does not work
vb.show()
vb2.show()

In [ ]:
from visbrain.gui import Brain
help(Brain)

## compare Jan's MNI grid with standard freesurfer brain

In [ ]:
import gc; gc.collect()

surf_type = 'pial' # 
#surf_type = 'white'
surf_type = 'inflated' # 
surfnames = [f'lh.{surf_type}', f'rh.{surf_type}']
#%debug
print(surfnames)
p = '/home/demitau/data/freesurfer_surf'
spaths = [pjoin(p,sn) for sn in surfnames]
tc = True
b1 = BrainObj(spaths, hemisphere='both' , translucent=tc ) 
b2 = BrainObj(spaths, hemisphere='both' , translucent=tc ) 


verts = b1.vertices
maxs = np.max( verts, axis=0)
mins = np.min( verts, axis=0)
difs = maxs - mins
#s_obj1.fit_to_vertices(verts) # result it shift

xyz =  cgMNI['headsurfgrid_verts'].T 
maxs2 = np.max( xyz, axis=0)
mins2 = np.min( xyz, axis=0)
difs2 = maxs2 - mins2
xyz -= ( (maxs2+mins2)/2 )[None,:]
xyz = xyz * ( (difs / difs2)[None,:] )
xyz += ( (maxs+mins)/2 )[None,:]

data = np.arange(xyz.shape[0])
radius_supp = 4
RADINFO = dict(radius_min=radius_supp, radius_max = radius_supp)

s_obj1  = SourceObj ('mysrc',xyz, data=data, cmap='inferno', **RADINFO)
s_obj2  = SourceObj ('mysrc',xyz, data=data, cmap='inferno', **RADINFO)


from visbrain.objects import BrainObj, SceneObj
bgcolor = 'black'
bgcolor = 'white'
sc = SceneObj(bgcolor=bgcolor, size=(400, 400))

#sc.add_to_subplot(b, row=0, col=0)
sc.add_to_subplot(b1, row=0, col=0, rotate='left')
sc.add_to_subplot(b2, row=0, col=1, rotate='top')
sc.add_to_subplot(s_obj1, row=0, col=0, rotate='left')
sc.add_to_subplot(s_obj2, row=0, col=1, rotate='top')

sc.preview()